# **Exploring $WZ$ Diboson Production with ATLAS Open Data**

This notebook guides you through the analysis of $WZ$ diboson production, specifically focusing on the final state with three leptons ($WZ → 3l$). Using ATLAS Open Data from proton-proton collisions at the Large Hadron Collider (LHC), you will apply event selection criteria to identify and analyze this process.

# ATLAS Open Data

ATLAS Open Data provides publicly available datasets recorded by the ATLAS experiment at the LHC. These datasets allow students and researchers to explore real high-energy physics data, applying analysis techniques similar to those used in professional research.

# What Are Notebooks?

Jupyter notebooks provide an interactive environment for combining live code execution, visualizations. This makes them an ideal platform for conducting and documenting particle physics analyses.

# The Goal: Identifying $WZ → 3l$ Events

In this analysis, we aim to reconstruct the $WZ$ process where:

* A $W$ boson decays into a lepton (electron or muon) and a neutrino.

* A $Z$ boson decays into a pair of same-flavor opposite-sign leptons.

# Running a Jupyter Notebook

To execute all cells, go to the top menu and select Cell -> Run All.

To run an individual cell, select Cell -> Run Cells or use Shift+Enter.

In [ ]:
!pip install uproot

# **Set up the notebook**


We're going to be using a number of tools to help us:

* uproot: lets us read .root files typically used in particle physics into data formats used in python

* awkward: lets us use efficiently the nested data in columnar format

* pandas: lets us store data as dataframes, a format widely used in python

* numpy: provides numerical calculations such as histogramming

* matplotlib: common tool for making plots, figures, images, visualisations

In [ ]:
import uproot
import awkward as ak
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator # for minor ticks
from scipy.optimize import curve_fit
import requests
import warnings

# Filter out any warnings that are expected, to avoid confusion while running
warnings.filterwarnings("ignore", message="invalid value encountered in sqrt")
warnings.filterwarnings("ignore", message="overflow encountered in power")
warnings.filterwarnings("ignore", message="overflow encountered in multiply", category=RuntimeWarning)
warnings.filterwarnings("ignore", message="invalid value encountered in subtract")

# **Explanation of Key Parameters**
In particle physics analyses, various parameters and constants are essential for accurately processing data and interpreting results.

* Integrated luminosity (lumi): is a measure of the total amount of data collected by a particle detector over a certain period. It represents the total number of potential collisions that could have occurred in a particle accelerator and is typically measured in inverse femtobarns .
Fraction of Events to Process

* fraction: this parameter controls what fraction of the available events in the dataset will be processed by the analysis in each iteration.

In [ ]:
lumi = 36000 # This sample is 36000 inverse picobarns
fraction = 0.1 #for 10 iterations, each one will process 10% of the total data

# **Defining the Sample Datasets**

To analyze $WZ$ production, we use both real and simulated (Monte Carlo) datasets. The datasets are categorized into different groups based on their physics processes:

* Data: Real collision data recorded by ATLAS.

* $WZ$ signal: Simulated WZ events, our main signal process.

* $WZ$ background (processes that can mimic WZ events):

  1.   Other diboson processes ($WW, ZZ$)
  2.   events from $Z+jets$ and $W+jets$
  3.   Top quark pair production and single top production


The following dictionary organizes these datasets for processing in the analysis.

In [ ]:
# Dictionary of samples to be processed
samples = {

    'data': {'list' : [
        'data15_periodJ',
        'data16_periodB',
        'data16_periodK',
        'data16_periodC',
        'data15_periodE',
        'data16_periodF',
        'data16_periodG',
        'data16_periodD',
        'data15_periodD',
        'data15_periodH',
        'data15_periodF',
        'data15_periodG',
        'data16_periodE',
        'data16_periodA',
        'data16_periodI',
        'data16_periodL',
        ],},



    # Main Monte Carlo (MC) sample: top quark pair production
    'wz' : {  'list' : ['mc_700601.Sh_2212_lllv'],},

    # Diboson WW and ZZ
    'diboson' : {  'list' : [
        'mc_700493.Sh_2211_ZqqZll',
        'mc_700488.Sh_2211_WlvWqq',
        'mc_700492.Sh_2211_WqqZll',
        'mc_700600.Sh_2212_llll',
        #'mc_700494.Sh_2211_ZbbZll',
        ],},

    # Z + jets and W + jets
    'vjets' : {  'list' : [
        'mc_700323.Sh_2211_Zmumu_maxHTpTV2_BFilter',
        'mc_700324.Sh_2211_Zmumu_maxHTpTV2_CFilterBVeto',
        'mc_700325.Sh_2211_Zmumu_maxHTpTV2_CVetoBVeto',
        'mc_700470.Sh_2211_Zmumu_maxHTpTV2_m10_40_pT5_BFilter',
        'mc_700471.Sh_2211_Zmumu_maxHTpTV2_m10_40_pT5_CFilterBVeto',
        'mc_700472.Sh_2211_Zmumu_maxHTpTV2_m10_40_pT5_CVetoBVeto',
        'mc_700320.Sh_2211_Zee_maxHTpTV2_BFilter',
        'mc_700321.Sh_2211_Zee_maxHTpTV2_CFilterBVeto',
        'mc_700322.Sh_2211_Zee_maxHTpTV2_CVetoBVeto',
        'mc_700467.Sh_2211_Zee_maxHTpTV2_m10_40_pT5_BFilter',
        'mc_700468.Sh_2211_Zee_maxHTpTV2_m10_40_pT5_CFilterBVeto',
        'mc_700469.Sh_2211_Zee_maxHTpTV2_m10_40_pT5_CVetoBVeto',
        'mc_700792.Sh_2214_Ztautau_maxHTpTV2_BFilter',
        'mc_700793.Sh_2214_Ztautau_maxHTpTV2_CFilterBVeto',
        'mc_700794.Sh_2214_Ztautau_maxHTpTV2_CVetoBVeto',

        'mc_700341.Sh_2211_Wmunu_maxHTpTV2_BFilter',
        'mc_700342.Sh_2211_Wmunu_maxHTpTV2_CFilterBVeto',
        'mc_700343.Sh_2211_Wmunu_maxHTpTV2_CVetoBVeto',
        'mc_700344.Sh_2211_Wtaunu_L_maxHTpTV2_BFilter',
        'mc_700345.Sh_2211_Wtaunu_L_maxHTpTV2_CFilterBVeto',
        'mc_700346.Sh_2211_Wtaunu_L_maxHTpTV2_CVetoBVeto',
        'mc_700347.Sh_2211_Wtaunu_H_maxHTpTV2_BFilter',
        'mc_700348.Sh_2211_Wtaunu_H_maxHTpTV2_CFilterBVeto',
        'mc_700349.Sh_2211_Wtaunu_H_maxHTpTV2_CVetoBVeto',
        'mc_700338.Sh_2211_Wenu_maxHTpTV2_BFilter',
        'mc_700339.Sh_2211_Wenu_maxHTpTV2_CFilterBVeto',
        'mc_700340.Sh_2211_Wenu_maxHTpTV2_CVetoBVeto',

        ],},

    # Single top quark production and ttbar samples
    'other' : {  'list' : [
        'mc_410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad',
        'mc_410644.PowhegPythia8EvtGen_A14_singletop_schan_lept_top',
        'mc_410645.PowhegPythia8EvtGen_A14_singletop_schan_lept_antitop',
        'mc_410658.PhPy8EG_A14_tchan_BW50_lept_top',
        'mc_410659.PhPy8EG_A14_tchan_BW50_lept_antitop',
        'mc_601355.PhPy8EG_tW_dyn_DR_incl_top',
        'mc_601352.PhPy8EG_tW_dyn_DR_incl_antitop',
        ],},

        }

# **Applying Event Weights in the Analysis**
To accurately compare simulated events to real collision data, we apply event weights. These weights account for various factors such as:

Pileup corrections: Adjustments for the modeling of additional collisions in the same event.

Lepton scale factors: Corrections for electron and muon identification, reconstruction, and trigger efficiencies.

Monte Carlo normalization: Scaling based on cross-sections, k-factors, and filtering efficiencies.

By applying event weights, we ensure that simulated samples accurately represent the expected event yields in real LHC data. The following function calculates the total event weight used in this analysis.

In [ ]:
def calc_weight(data):

    scale_factors = (data["ScaleFactor_PILEUP"] * data["ScaleFactor_ELE"] * data["ScaleFactor_MUON"] * data["ScaleFactor_MLTRIGGER"])

    norm_factor = (data["xsec"] * data["kfac"] * data["filteff"]) / data["sum_of_weights"]

    weight = data["mcWeight"] * scale_factors * norm_factor * lumi

    return weight

# **Applying Dilepton Trigger Selection in the Analysis**
In high-energy physics experiments like ATLAS, triggers play a crucial role in selecting events of interest while efficiently handling large data volumes.

For this analysis, we apply dilepton triggers, which are designed to fire when an event contains at least two leptons (electrons or muons) that pass certain kinematic and identification criteria. We consider two categories of dilepton triggers:

* Electron-based dilepton triggers (trigDE): Fire when two electrons satisfy the trigger conditions.

* Muon-based dilepton triggers (trigDM): Fire when two muons satisfy the trigger conditions.

In [ ]:
def cut_trig(trigDE,trigDM):
    return trigDE | trigDM

# **Matching Leptons to Dilepton Triggers**
In this analysis, we use dilepton triggers, which fire when an event contains at least two leptons that satisfy specific selection criteria. However, not all reconstructed leptons in an event necessarily contributed to the trigger decision. To ensure that at least one of the selected leptons was responsible for firing the trigger, we apply a trigger-matching condition.

The variable trigML indicates whether a lepton is matched to the dilepton trigger:

* trigML = 1 → The lepton is matched to the dilepton trigger.

* trigML = 0 → The lepton is not matched to the trigger.

To ensure that we only select events where at least one lepton was responsible for firing the dilepton trigger, we apply the following condition:

In [ ]:
def cut_match_MLtrig(trigML):
    return trigML==1


# **Selecting Events with Exactly Three Leptons**
In the $WZ → 3l$ analysis, we focus on events containing exactly three leptons. This requirement is crucial because:

The $W$ boson decays into a lepton (electron or muon) and a neutrino.

The $Z$ boson decays into a pair of same-flavor, opposite-sign (SFOS) leptons.

By enforcing this selection, we remove events with fewer or more leptons, which are unlikely to originate from $WZ$ production. The function below ensures that only events with exactly three leptons are retained for further analysis.

In [ ]:
def three_lep(lep_n):
    return lep_n == 3

# **Missing Transverse Energy ($MET$) Requirement**
In $WZ → 3l$ events, the W boson decays into a lepton and a neutrino. Since neutrinos do not interact with the ATLAS detector, they escape detection, leading to missing transverse energy.

To ensure that selected events contain a neutrino, we apply a minimum $MET$ threshold. This cut helps reduce backgrounds from processes without genuine neutrinos, such as $Z+jets$, where $MET$ arises from mismeasurements.

The function below applies a $MET$ cut to retain events with significant missing energy, improving the purity of the $WZ$ selection.

In [ ]:
def cut_met_et(met_et):
    return met_et > 3

# **Lepton Transverse Momentum ($p_T$) Selection**
Transverse momentum ($p_T$) is a crucial variable in event selection, as it helps ensure that the leptons are well-reconstructed and distinguishable from, low-energy background particles.

For the $WZ → 3l$ analysis, we apply two conditions to the lepton pt:

  1. At least three leptons must have $p_T > 20 GeV$ to guarantee all leptons are energetic enough for accurate reconstruction.

  2. At least one lepton must have $p_t > 25 GeV$ to ensure a high-momentum lepton, improving trigger efficiency.

These cuts enhance the signal purity while maintaining high efficiency for $WZ$ event selection. The function below implements these requirements.

In [ ]:
def cut_lep_pt(lep_pt):
    pt = lep_pt
    cut1 = ak.sum(pt > 20, axis=1) >= 3
    cut2 = ak.sum(pt > 25, axis=1) >= 1
    return cut1 & cut2

# **Lepton Identification and Isolation Requirement**
Lepton Isolation Selection (Track and Calorimeter)

In electroweak processes like $WZ → 3ℓ$, leptons are typically isolated from other particle activity,
whereas background leptons (e.g. from heavy flavor decays or jets faking leptons) are often surrounded by nearby tracks or energy deposits. To distinguish these, we apply two types of isolation cuts:
 1. **Track Isolation** (`pt_cone`): Measures the sum of transverse momenta of tracks around the lepton.
 2. **Calorimeter Isolation** (`et_cone`): Measures the sum of transverse energy deposits in the calorimeter near the lepton.

For both types, we use the ratio of isolation energy to lepton transverse momentum (`iso / pt`).
A ratio below 0.15 indicates the lepton is well-isolated.
In each case, the event is kept only if **more than two leptons** satisfy `iso / pt < 0.15`.
This ensures that all three leptons in the WZ → 3ℓ event are likely prompt, clean, and isolated.


In [ ]:
def cut_track_iso(pt_cone, pt):
    cut = ak.sum(np.abs(pt_cone/pt) < 0.15, axis=1) > 2
    return cut

In [ ]:
def cut_cal_iso(et_cone, pt):
    cut = ak.sum(np.abs(et_cone/pt) < 0.15, axis=1) > 2
    return cut

# **Lepton Selection: Tight ID with Loose Isolation**

In this step, we apply a combined selection that uses:

  1. **Tight Identification (ID):** Ensures leptons are of high quality — consistent with prompt,
     well-reconstructed leptons from the primary vertex (not fakes or from hadron decays).
  2. **Loose Isolation (iso):** Allows some nearby detector activity around the lepton.

This improves efficiency (especially in busy events) while still rejecting most background.

The function checks whether each lepton satisfies both conditions (`ID & iso`) and then counts how many such leptons exist in each event.

The event is selected only if **more than two leptons** pass both tight ID and loose isolation. This is a common strategy in WZ → 3ℓ analyses, where the goal is to retain high signal efficiency without compromising much on background rejection.


In [ ]:
def ID_iso_cut(ID,iso):
    return ak.sum(ID & iso,axis=1) > 2

# **Same-Flavor Opposite-Sign (SFOS) Lepton Pair Selection**
In the $WZ → 3ℓ$ analysis, identifying a same-flavor opposite-sign (SFOS) lepton pair is crucial for reconstructing the Z boson. The $Z$ boson decays into an electron-positron ($e^-e^+$) or muon-antimuon ($μ^-μ^+$) pair, so we need to:


*   Select lepton pairs with the same flavor (both electrons or both muons)
*   Ensure the selected pair has opposite charges (one positively and one negatively charged lepton)

This step is essential to distinguish real $Z$ boson decays from background processes where lepton pairs may arise from other sources. The function below applies these conditions to each event to retain only those with at least one valid SFOS lepton pair.

In [ ]:
def sfos_pair(flavor, charge):

    # Generate all unique pairs of particles for each event
    # Combinations of 2 ensure no particle is compared with itself
    f_pairs = ak.combinations(flavor, 2, fields=["flavor_0", "flavor_1"])
    c_pairs = ak.combinations(charge, 2, fields=["charge_0", "charge_1"])

    # Same-flavor check: both particles in the pair must be of the same type
    same_flavor = f_pairs.flavor_0 == f_pairs.flavor_1

    # Opposite-sign check: the charges of the particles must be different
    opposite_sign = c_pairs.charge_0 != c_pairs.charge_1

    # Find where both conditions are met
    sfos = same_flavor & opposite_sign

    # We want at least one SFOS pair per event
    sfos_cut = ak.any(sfos, axis=1)

    return sfos_cut

# **Lepton Four-Momentum Packaging**
For the WZ → 3ℓ analysis, we need to package the leptons into four-momentum vectors to facilitate further calculations, such as reconstructing invariant masses and analyzing kinematic variables. The four-momentum of a particle is defined by its energy ($E$) and momentum components ($p_x, p_y, p_z$).

The function below converts the lepton's transverse momentum ($p_t$), pseudorapidity ($η$), and azimuthal angle ($φ$) into the full momentum components in three dimensions ($p_x, p_y, p_z$), and packages this information into a structured format that can be used for further analysis. It also includes the missing transverse energy ($MET$) and $MET$ $phi$, which are important for the $W$ boson reconstruction.

In [ ]:
def package_leptons(flavor, charge, E, pt, eta, phi, met, met_phi):
    lep_px = pt * np.cos(phi)
    lep_py = pt * np.sin(phi)
    lep_pz = pt / np.tan(2.0 * np.arctan(np.exp(-eta)))

    leptons = ak.zip({
        "flavor": flavor,
        "charge": charge,
        "E": E,
        "pt": pt,
        "eta": eta,
        "phi": phi,
        "px": lep_px,
        "py": lep_py,
        "pz": lep_pz,
        "met": met,
        "met_phi": met_phi,
    })

    return leptons

# **Finding the Closest SFOS Pair to the $Z$ Boson Mass**
Each event may have more than one same-flavor opposite-sign (SFOS) lepton pairs , so selecting the correct SFOS pair that reconstructs the $Z$ boson mass is a critical step. The function below identifies the lepton pair closest to the $Z$ boson mass (around $91 GeV$) from a set of SFOS pairs. This is done by calculating the invariant mass of each pair and selecting the one with the smallest difference from the $Z$ mass.

The process involves:


1.   Creating lepton pairs: All unique pairs of leptons from the event are generated.
2.   Applying the SFOS condition: Only pairs that are of the same flavor (e.g., both electrons or both muons) and opposite sign (i.e., opposite charges) are kept.
3. Calculating the invariant mass of each SFOS pair: The four-momentum is used to calculate the invariant mass of each pair.
4. Finding the pair closest to the $Z$ boson mass: The mass difference between the calculated invariant mass of each SFOS pair and the nominal $Z$ boson mass is calculated. The pair with the smallest mass difference is selected.
5. Returning the selected leptons: The two leptons that form the SFOS pair closest to the $Z$ mass, along with the invariant mass of the pair, are returned.



In [ ]:
def find_closest_sfos_pair(leptons):
    z_mass = 91.0
    z_window = 10.0


    # Create all unique lepton pairs
    lep_pairs = ak.combinations(leptons, 2, fields=["lep1", "lep2"])

    # SFOS condition
    same_flavor = lep_pairs.lep1.flavor == lep_pairs.lep2.flavor
    opposite_sign = lep_pairs.lep1.charge != lep_pairs.lep2.charge
    sfos_mask = same_flavor & opposite_sign

    # Filter SFOS pairs
    sfos_pairs = lep_pairs[sfos_mask]

    # Compute invariant mass of SFOS pairs
    E_sum = sfos_pairs.lep1.E + sfos_pairs.lep2.E
    px_sum = sfos_pairs.lep1.px + sfos_pairs.lep2.px
    py_sum = sfos_pairs.lep1.py + sfos_pairs.lep2.py
    pz_sum = sfos_pairs.lep1.pz + sfos_pairs.lep2.pz

    masses = np.sqrt(E_sum**2 - (px_sum**2 + py_sum**2 + pz_sum**2))

    # Compute absolute difference from Z mass
    mass_diff = np.abs(masses - z_mass)

    # Create a mask for the pair closest to the Z boson mass
    closest_mask = mass_diff == ak.min(mass_diff, axis=1)

    # Use boolean mask to select the closest SFOS pair
    closest_pair = sfos_pairs[closest_mask]
    closest_mass = masses[closest_mask]

    selected_lep1 = closest_pair.lep1
    selected_lep2 = closest_pair.lep2


    return selected_lep1, selected_lep2, closest_mass

# **Z Boson Mass Cut**
In the analysis of the $WZ → 3l$ process, we need to identify the leptons that are consistent with the decay of a $Z$ boson. The $Z$ boson decays into a pair of same-flavor, opposite-sign leptons, and we expect the invariant mass of this lepton pair to be close to the known mass of the $Z$ boson (around $91 GeV$).  This uses the same 10 GeV mass window as the previous function.

To ensure we are selecting lepton pairs consistent with this decay, we apply a cut to select only those pairs whose mass falls within the typical $Z$ boson mass window ($81 GeV < mass < 101 GeV$).

This cut is crucial for rejecting background events and selecting signal events where the invariant mass of the lepton pair matches the expected mass of the $Z$ boson. The function returns a boolean array indicating whether each event passes this mass cut.

In [ ]:
def z_boson_cut(closest_mass):
    # Z mass window cut: 81 < mass < 101
    z_mass_cut_mask = (closest_mass > 81) & (closest_mass < 101)
    z_mass_cut = ak.any(z_mass_cut_mask, axis=1)
    return z_mass_cut

# **Splitting SFOS Pairs by flavor**




After selecting the closest same-flavor opposite-sign (SFOS) lepton pair, it's useful to categorize these pairs by their lepton flavor (electron or muon). This step ensures we handle the different lepton flavors separately.

This function splits the closest SFOS pairs into two categories:

1. Electron pairs ($ee$): Pairs where both leptons are electrons (flavor code 11).

2. Muon pairs ($μμ$): Pairs where both leptons are muons (flavor code 13).

The function performs the following:

* It checks whether the lepton pair consists of two electrons or two muons.


* It then creates separate arrays for the invariant mass of the SFOS pairs, depending on whether the pair is electron-electron ($e^-e^+$) or muon-muon ($μ^-μ^+$).




In [ ]:
def split_sfos_pairs(paired_lep1, paired_lep2, closest_mass):

    is_ee = (paired_lep1.flavor == 11) & (paired_lep2.flavor == 11)
    is_mm = (paired_lep1.flavor == 13) & (paired_lep2.flavor == 13)

    closest_ee_mass = ak.mask(closest_mass, is_ee)
    closest_mumu_mass = ak.mask(closest_mass, is_mm)

    return closest_ee_mass, closest_mumu_mass

# **Identifying the Unpaired Lepton**
In events with exactly three leptons, we need to identify which lepton is unpaired, meaning it is not part of the selected same-flavor opposite-sign (SFOS) pair. This step is crucial for reconstructing the $W$ boson, as the unpaired lepton corresponds to the decay product of the $W$ boson, while the SFOS pair corresponds to the $Z$ boson.

The function operates as follows:

1. It extracts the individual leptons from the set of three leptons in the event.

2. It then checks which lepton is not part of the selected SFOS pair by comparing the transverse momentum ($p_T$) of each lepton to those of the two paired leptons.

3. The lepton that is not part of the SFOS pair is identified as the unpaired lepton.

The function uses the ak.where function to apply these conditions and returns the unpaired lepton, which will later be used for reconstructing the $W$ boson mass.

In [ ]:
def find_unpaired_lepton(leptons, paired_lep1, paired_lep2):
    # Extract individual leptons
    lep1, lep2, lep3 = leptons[:, 0], leptons[:, 1], leptons[:, 2]

    # Identify the unpaired lepton
    unpaired_lepton = ak.where(
        (lep1.pt != paired_lep1.pt) & (lep1.pt != paired_lep2.pt), lep1,
        ak.where((lep2.pt != paired_lep1.pt) & (lep2.pt != paired_lep2.pt), lep2, lep3)
    )

    return unpaired_lepton

# **Calculating the Transverse Mass of the W Boson**
The transverse mass of the W boson is a key quantity in reconstructing the W boson in $WZ → 3l$ events. The full momentum of the neutrino is not known. Through momentum balance, its transverse momentum (momentum in x and y) can be determined from the rest of the event. The transverse mass uses this transverse momentum instead of the full momentum vector to calculate a mass. Instead of having a sharp peak at the mass of the W boson, it is expected to have a broader distribution that falls off sharply around the mass of the W boson. This function computes the transverse mass using the unpaired lepton and missing transverse energy ($MET$).

  The transverse mass is calculated using the formula:
  $m_T^2 = \left( p_T^{\text{lep}} + p_T^{\text{MET}} \right)^2 - \left( p_X^{\text{lep}} + p_X^{\text{MET}} \right)^2 - \left( p_Y^{\text{lep}} + p_Y^{\text{MET}} \right)^2$



In [ ]:
def w_boson_mass(unpaired_lepton):

    # Convert from MeV to GeV
    lep_pt = unpaired_lepton.pt
    lep_phi = unpaired_lepton.phi
    lep_px = unpaired_lepton.px
    lep_py = unpaired_lepton.py
    met_et = unpaired_lepton.met
    met_phi = unpaired_lepton.met_phi
    # Calculate the transverse components (px and py) of the lepton and MET
    lep_px = lep_pt * np.cos(lep_phi)
    lep_py = lep_pt * np.sin(lep_phi)
    met_px = met_et * np.cos(met_phi)
    met_py = met_et * np.sin(met_phi)

    # Calculate the transverse mass of the W boson
    w_mt_squared = ((lep_pt + met_et)**2 - ((lep_px + met_px)**2 + (lep_py + met_py)**2))
    w_mt_squared = np.maximum(w_mt_squared, 0)  # Set negative values to zero
    w_mt = np.sqrt(w_mt_squared)

    return w_mt

# **W boson mass cut**
The function below filters out events where the transverse mass is below a certain threshold ($30 GeV$). This cut ensures that only events with a significant transverse mass are retained for further analysis, improving the signal-to-background ratio.

In [ ]:
def w_boson_mass_cut(w_mt):

    w_mass_cut_mask = w_mt > 30
    w_mass_cut = ak.any(w_mass_cut_mask, axis=1)

    return w_mass_cut

# **Splitting the W boson by lepton type**
 The function below splits the transverse mass values ($M_T^W$) into two arrays: one for events with a muon as the unpaired lepton and one for events with an electron. This separation allows for more precise analysis of each channel (muon and electron), as their characteristics and response in the detector are distinct.



In [ ]:
def split_by_type(w_mt, unpaired_lepton):

    # Create masks for muons and electrons
    is_muon = unpaired_lepton.flavor == 13
    is_electron = unpaired_lepton.flavor == 11

    # Use the mask to filter w_mt for muons and electrons
    w_mt_muon = ak.mask(w_mt, is_muon)
    w_mt_electron = ak.mask(w_mt, is_electron)

    return w_mt_muon, w_mt_electron

# **Data Processing and Event Selection Function**

The read_file function is designed to read and process data from a file, apply selection cuts, and return an awkward array containing the events that pass all cuts.

In [ ]:
def read_file(path,sample,loop):
    data_all = [] # define empty list to hold all data for this sample

    # open the tree using uproot
    with uproot.open(path + f":analysis;1") as tree:
        numevents = tree.num_entries
        # Loop over the fraction of events requested above using all the variables we need for the analysis
        for data in tree.iterate(
            [
                "mcWeight", "sum_of_weights", "ScaleFactor_ELE", "ScaleFactor_MUON", "ScaleFactor_MLTRIGGER",
                "ScaleFactor_PILEUP", "xsec", "filteff", "kfac", "trigDE", "trigDM", "TriggerMatch_DILEPTON",
                "lep_n", "lep_pt", "lep_eta", "lep_phi", "lep_charge", "lep_type", "met", "met_phi",
                "lep_e", "lep_isTightID", "lep_isLooseIso", "lep_topoetcone20", "lep_ptvarcone30",
            ],
            library="ak",
            entry_start=int(round(numevents * fraction * loop, 0)),
            entry_stop=int(round(numevents * fraction * (loop + 1), 0))
        ):

            # Apply cuts step by step, updating counters as we go
            data = data[cut_trig(data.trigDE, data.trigDM)]

            data = data[cut_match_MLtrig(data.TriggerMatch_DILEPTON)]

            data = data[three_lep(data.lep_n)]

            data = data[cut_met_et(data.met)]

            data = data[cut_lep_pt(data.lep_pt)]

            data = data[ID_iso_cut(data.lep_isTightID, data.lep_isLooseIso)]

            data = data[cut_cal_iso(data.lep_topoetcone20, data.lep_pt)]

            data = data[cut_track_iso(data.lep_ptvarcone30, data.lep_pt)]


            leptons = package_leptons(data.lep_type, data.lep_charge, data.lep_e, data.lep_pt, data.lep_eta, data.lep_phi, data.met, data.met_phi)
            sfos_pairs, sfos_mask = find_sfos_pairs(leptons)
            sfos_cut = sfos_cut(sfos_mask)
            data = data[sfos_cut]
            leptons = leptons[sfos_cut]
            sfos_pairs = sfos_pairs[sfos_cut]

            paired_lep1, paired_lep2, closest_mass = find_closest_sfos_pair(sfos_pairs)
            z_mass_cut = z_mass_cut(closest_mass)

            paired_lep1 = paired_lep1[z_mass_cut]
            paired_lep2 = paired_lep2[z_mass_cut]
            closest_mass = closest_mass[z_mass_cut]

            ee_mass, mumu_mass = split_sfos_pairs(paired_lep1, paired_lep2, closest_mass)
            unpaired_lepton = find_unpaired_lepton(leptons, paired_lep1, paired_lep2)
            w_mass_cut, w_mt = w_boson_mass(unpaired_lepton)
            w_mt_muon, w_mt_electron = split_by_type(w_mt, unpaired_lepton)

            data = data[w_mass_cut]

            data['wtmass'] = w_mt[w_mass_cut]
            data['zmass'] = closest_mass[w_mass_cut]
            data['ee_pair_mass'] = ee_mass[w_mass_cut]
            data['mumu_pair_mass'] = mumu_mass[w_mass_cut]
            data['wmuon'] = w_mt_muon[w_mass_cut]
            data['welectron'] = w_mt_electron[w_mass_cut]

            # Compute weights
            if 'data' not in sample:
                data['weight'] = calc_weight(data)
            else:
                data['weight'] = ak.zeros_like(data['met'])

            data_all.append(data)
            del data, z_mass_cut, w_mass_cut  # Free up memory

    return ak.concatenate(data_all)

# **Data Retrieval in $WZ \rightarrow 3l$ Analysis**
This function is essential for managing the various datasets needed for a thorough analysis. It focuses on actual experimental data collected from proton-proton collisions at the Large Hadron Collider (LHC).

In [ ]:
def get_data_from_files(type_data, loop):

    data = [] # define empty list to hold data
    for val in samples[type_data]['list']: # loop over each file
        if type_data == "data":
            prefix = "Data/"
        else: prefix = "MC/"
        tuple_path = "root://eosuser.cern.ch//eos/user/e/egramsta/OpenData/FEB2025/exactly3lep/"
        fileString = tuple_path+prefix+val+".exactly3lep.root"
        data.append(read_file(fileString,val,loop))

    return ak.concatenate(data)

# **Data Processing**
The analysis function is a versatile tool designed to process data from specific datasets (data_type) in distinct iterations (loop). It efficiently extracts, organizes, and returns data in the form of a Pandas DataFrame, making it ready for further aggregation and analysis.

In [ ]:
def analysis(data_type,loop):

    # Process data for data_type sample
    data = get_data_from_files(data_type,loop)
    data_df = pd.DataFrame({
            "Weight": ak.to_list(data['weight']),
            "missing_Et": ak.to_list(data['met']),
            "W_tmass": ak.to_list(data['wtmass']),
            "W_muon": ak.to_list(data['wmuon']),
            "w_electron": ak.to_list(data['welectron']),
            "Z_mass": ak.to_list(data['zmass']),
            "ee_mass": ak.to_list(data['ee_pair_mass']),
            "mumu_mass": ak.to_list(data['mumu_pair_mass']),

        })

    del(data)       # Delete the 'data' dictionary to free up memory
    return data_df  # Return the created Pandas DataFrame

#**Looped Data Processing & Performance Tracking**
This block executes the full analysis in 10 sequential steps, each handling 10% of the dataset (defined by fraction = 0.1).

In each iteration, the analysis function is called to process a specific chunk, and the resulting DataFrame is appended to the main DataFrame using pd.concat().

The start and end time of each iteration are tracked using the time module to monitor performance and give feedback on the runtime.

This modular looped approach helps manage memory and enables progress tracking through printed updates showing both time elapsed and percentage of total data processed.


In [ ]:
loops = 10             # reduce this if you want the code to run quicker (1-10)
start = time.time()  # Time at the start of the whole processing
main_analysis_df = pd.DataFrame()

for i in range(loops):
    main_analysis_df = pd.concat([analysis("data", i), main_analysis_df], ignore_index=True)

elapsed = time.time() - start
print(f"Time taken: {round(elapsed/60,1)} min")

In the previous cell we handled the real data samples, so in this cell we follow the same process but for the Monte Carlo samples.

In [ ]:
loops = 10
start = time.time()

wz_data_df = pd.DataFrame()
diboson_data_df = pd.DataFrame()
vjets_data_df = pd.DataFrame()
other_data_df = pd.DataFrame()

for i in range(loops):
    wz_data_df = pd.concat([analysis("wz", i), wz_data_df], ignore_index=True)
    diboson_data_df = pd.concat([analysis("diboson", i), diboson_data_df], ignore_index=True)
    vjets_data_df = pd.concat([analysis("vjets", i), vjets_data_df], ignore_index=True)
    other_data_df = pd.concat([analysis("other", i), other_data_df], ignore_index=True)

elapsed = time.time() - start
print(f"Time taken: {round(elapsed/60,1)} min")

#**Plotting**

In [ ]:
def plot_data(data, xmin, xmax, step_size, x_label, y_label, data_mc, mc_Weight, data_di,
              di_Weight, data_vjets, vjets_Weight, data_other, other_Weight):
    # Define MC data sets and their properties
    datasets = [
        {'data': data_vjets, 'weights': vjets_Weight, 'color': 'green', 'label': r'V+jets'},
        {'data': data_other, 'weights': other_Weight, 'color': 'orange', 'label': r'OTHER'},
        {'data': data_di, 'weights': di_Weight, 'color': 'slateblue', 'label': r'ZZ,WW'},
        {'data': data_mc, 'weights': mc_Weight, 'color': 'lightcoral', 'label': r'WZ'},
    ]

    # Create bin edges
    bin_edges = np.arange(xmin, xmax + step_size, step_size)

    # Convert data to numpy for histogramming
    data_np = ak.to_numpy(data)

    # Handle overflow: add events > xmax to the last bin
    data_np[data_np > xmax] = xmax

    # Histogram the data
    data_x, _ = np.histogram(data_np, bins=bin_edges)
    data_x_errors = np.sqrt(data_x)  # Statistical error on the data

    # Mask bins with zero events
    mask_nonzero = data_x > 0
    bin_centres = (bin_edges[:-1] + bin_edges[1:]) / 2
    masked_centres = bin_centres[mask_nonzero]
    masked_data_x = data_x[mask_nonzero]
    masked_data_x_errors = data_x_errors[mask_nonzero]

    # Create main plot and residual subplot
    fig, (main_axes, residual_axes) = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)

    # Plot data with error bars, excluding bins with zero events
    main_axes.errorbar(x=masked_centres, y=masked_data_x, yerr=masked_data_x_errors,
                       fmt='ko', label='Data')

    # Adjust MC datasets for overflow handling
    for dataset in datasets:
        dataset['data'] = ak.to_numpy(dataset['data'])
        dataset['data'][dataset['data'] > xmax] = xmax

    # Plot the Monte Carlo bars
    mc_heights = main_axes.hist([d['data'] for d in datasets], bins=bin_edges,
                                weights=[d['weights'] for d in datasets], stacked=True,
                                color=[d['color'] for d in datasets], label=[d['label'] for d in datasets])

    # Extract the total stacked MC heights from the last array in mc_heights[0]
    mc_x_tot = mc_heights[0][-1]  # Total height in each bin after stacking

    # Set up main axes
    main_axes.set_xlim(left=xmin, right=xmax)
    ymax = max(np.max(data_x), np.max(mc_x_tot))
    main_axes.set_ylim(0, ymax * 1.4)  # Add 40% headspace

    main_axes.xaxis.set_minor_locator(AutoMinorLocator())
    main_axes.tick_params(which='both', direction='in', top=True, right=True)
    main_axes.set_ylabel(y_label, y=1, horizontalalignment='right')
    main_axes.yaxis.set_minor_locator(AutoMinorLocator())

    # Add text to the plot
    main_axes.text(0.05, 0.93, 'ATLAS Open Data', transform=main_axes.transAxes, fontsize=13)
    main_axes.text(0.05, 0.88, 'for education', transform=main_axes.transAxes, style='italic', fontsize=8)
    main_axes.text(0.05, 0.82, r'$\sqrt{s}$=13 TeV, 36 fb$^{-1}$', transform=main_axes.transAxes)
    main_axes.text(0.05, 0.76, r'$W Z \rightarrow \ell \ell \ell ν$', transform=main_axes.transAxes)

    main_axes.legend(frameon=False)

    # Calculate and plot residuals
    ratio = data_x / mc_x_tot  # Residuals: Data / Total Stacked MC
    residual_axes.errorbar(bin_centres, ratio, yerr=ratio * data_x_errors / data_x, fmt='ko')
    residual_axes.axhline(1, color='r', linestyle='--')
    residual_axes.set_ylim(0, 2)
    residual_axes.set_xlabel(x_label, fontsize=13, x=1, horizontalalignment='right')
    residual_axes.set_ylabel('Ratio (Data/MC)')
    residual_axes.xaxis.set_minor_locator(AutoMinorLocator())
    residual_axes.yaxis.set_minor_locator(AutoMinorLocator())
    residual_axes.tick_params(which='both', direction='in', top=True, right=True)

    # Adjust layout
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.05)

    plt.show()


# **Mass and Kinematic Distributions**

This section prepares and visualizes several key physical observables from:
- **main_analysis_df**: Real data (observed events from the detector)
- **wz_data_df**: Monte Carlo simulation of the signal process (W±Z → 3 leptons)
- **diboson_data_df**, **vjets_data_df**, **other_data_df**: Background MC samples

These plots help validate event selection and study discriminating features
between data, expected signal, and background processes.

Some physics quantities are stored as nested Awkward Arrays and must be flattened before plotting, so we use:
1. `ak.flatten()` to remove nesting
2. and `ak.to_numpy()` to convert to NumPy arrays

This is done per dataset to make them compatible with `np.hstack()` and plotting. Other variables like ("missing_Et", "W_tmass", "Z_mass") are already stored as flat arrays and do not require flattening.


The `plot_data` function is then called to produce histograms for the following:

1. **Missing Transverse Energy (MET)**:
  - Variable: "missing_Et"
  - Range: [0, 160] GeV, 8 bins
  - X-axis: "$E_T^{\mathrm{miss}}$ [GeV]"
2. **Transverse Mass of W Boson (from lepton + MET)**:
  - Variable: "W_tmass"
  - Range: [0, 200] GeV, 10 GeV bins
  - X-axis: "$M_T^{\mathrm{W}}$ [GeV]"

3. **Invariant Mass of Z Candidate from SFOS Lepton Pair**:
  - Variable: "Z_mass"
  - Range: [75, 105] GeV, 1.5 GeV bins
  - X-axis: "$M_{ll}$ [GeV]"

4. **W Boson Mass from Muon Channel (W → μν)**:
  - Variable: "w_muon"
  - Range: [0, 200] GeV, 10 GeV bins
  - X-axis: "$W_μ^{\mathrm{mass}}$ [GeV]"

5. **W Boson Mass from Electron Channel (W → eν)**:
  - Variable: "w_electron"
  - Range: [0, 200] GeV, 10 GeV bins
  - X-axis: "$W_e^{\mathrm{mass}}$ [GeV]"

6. **Z Boson Mass from Muon Pairs (μμ)**:
  - Variable: "mumu_mass"
  - Range: [75, 105] GeV, 1.5 GeV bins
  - X-axis: "$M_{μμ}$ [GeV]"

7. **Z Boson Mass from Electron Pairs (ee)**:
  - Variable: "ee_mass"
  - Range: [75, 105] GeV, 1.5 GeV bins
  - X-axis: "$M_{ee}$ [GeV]"

Notes:
- The mc data are scaled using event weights.
- These distributions are crucial for validating selection criteria and studying physics objects (W and Z bosons) in different channels.



In [ ]:
# Flatten the columns before passing them
plot_data(
    np.hstack(main_analysis_df["missing_Et"]), 0, 160, 8, r"$E_T^{\mathrm{miss}}$ [GeV]", "events / bin",
    np.hstack(wz_data_df["missing_Et"]), np.hstack(wz_data_df['Weight']),
    np.hstack(diboson_data_df["missing_Et"]), np.hstack(diboson_data_df['Weight']),
    np.hstack(vjets_data_df["missing_Et"]), np.hstack(vjets_data_df['Weight']),
    np.hstack(other_data_df["missing_Et"]), np.hstack(other_data_df['Weight'])
)

In [ ]:
plot_data(
    np.hstack(main_analysis_df["W_tmass"]), 0, 200, 10, r"$M_T^{\mathrm{W}}$ [GeV]", "events / bin",
    np.hstack(wz_data_df["W_tmass"]), np.hstack(wz_data_df['Weight']),
    np.hstack(diboson_data_df["W_tmass"]), np.hstack(diboson_data_df['Weight']),
    np.hstack(vjets_data_df["W_tmass"]), np.hstack(vjets_data_df['Weight']),
    np.hstack(other_data_df["W_tmass"]), np.hstack(other_data_df['Weight'])
)

plot_data(
    np.hstack(main_analysis_df["Z_mass"]), 75, 105, 1.5, r"$M_ll$ [GeV]", 'events / bin',
    np.hstack(wz_data_df["Z_mass"]), np.hstack(wz_data_df['Weight']),
    np.hstack(diboson_data_df["Z_mass"]), np.hstack(diboson_data_df['Weight']),
    np.hstack(vjets_data_df["Z_mass"]), np.hstack(vjets_data_df['Weight']),
    np.hstack(other_data_df["Z_mass"]), np.hstack(other_data_df['Weight'])
)

In [ ]:
main_analysis_df["w_muon"] = ak.to_numpy(ak.flatten(main_analysis_df["w_muon"]))
wz_data_df["w_muon"] = ak.to_numpy(ak.flatten(wz_data_df["w_muon"]))
diboson_data_df["w_muon"] = ak.to_numpy(ak.flatten(diboson_data_df["w_muon"]))
vjets_data_df["w_muon"] = ak.to_numpy(ak.flatten(vjets_data_df["w_muon"]))
other_data_df["w_muon"] = ak.to_numpy(ak.flatten(other_data_df["w_muon"]))

plot_data(
    np.hstack(main_analysis_df["w_muon"]),  # Flatten Z_mass column
    0, 200, 10, r"$W_μ^{\mathrm{mass}}$ [GeV]", 'events / bin',
    np.hstack(wz_data_df["w_muon"]), np.hstack(wz_data_df['Weight']),
    np.hstack(diboson_data_df["w_muon"]), np.hstack(diboson_data_df['Weight']),
    np.hstack(vjets_data_df["w_muon"]), np.hstack(vjets_data_df['Weight']),
    np.hstack(other_data_df["w_muon"]), np.hstack(other_data_df['Weight'])
)

In [ ]:
main_analysis_df["w_electron"] = ak.to_numpy(ak.flatten(main_analysis_df["w_electron"]))
wz_data_df["w_electron"] = ak.to_numpy(ak.flatten(wz_data_df["w_electron"]))
diboson_data_df["w_electron"] = ak.to_numpy(ak.flatten(diboson_data_df["w_electron"]))
vjets_data_df["w_electron"] = ak.to_numpy(ak.flatten(vjets_data_df["w_electron"]))
other_data_df["w_electron"] = ak.to_numpy(ak.flatten(other_data_df["w_electron"]))

plot_data(
    np.hstack(main_analysis_df["w_electron"]),  # Flatten Z_mass column
    0, 200, 10, "$W_e^{\mathrm{mass}}$ [GeV]", 'events / bin',
    np.hstack(wz_data_df["w_electron"]), np.hstack(wz_data_df['Weight']),
    np.hstack(diboson_data_df["w_electron"]), np.hstack(diboson_data_df['Weight']),
    np.hstack(vjets_data_df["w_electron"]), np.hstack(vjets_data_df['Weight']),
    np.hstack(other_data_df["w_electron"]), np.hstack(other_data_df['Weight'])
)

In [ ]:
main_analysis_df["mumu_mass"] = ak.to_numpy(ak.flatten(main_analysis_df["mumu_mass"]))
wz_data_df["mumu_mass"] = ak.to_numpy(ak.flatten(wz_data_df["mumu_mass"]))
diboson_data_df["mumu_mass"] = ak.to_numpy(ak.flatten(diboson_data_df["mumu_mass"]))
vjets_data_df["mumu_mass"] = ak.to_numpy(ak.flatten(vjets_data_df["mumu_mass"]))
other_data_df["mumu_mass"] = ak.to_numpy(ak.flatten(other_data_df["mumu_mass"]))





plot_data(
    np.hstack(main_analysis_df["mumu_mass"]),  # Flatten Z_mass column
    75, 105, 1.5, "$M_{μμ}$ [GeV]", 'events / bin',
    np.hstack(wz_data_df["mumu_mass"]), np.hstack(wz_data_df['Weight']),
    np.hstack(diboson_data_df["mumu_mass"]), np.hstack(diboson_data_df['Weight']),
    np.hstack(vjets_data_df["mumu_mass"]), np.hstack(vjets_data_df['Weight']),
    np.hstack(other_data_df["mumu_mass"]), np.hstack(other_data_df['Weight'])
)

In [ ]:
main_analysis_df["ee_mass"] = ak.to_numpy(ak.flatten(main_analysis_df["ee_mass"]))
wz_data_df["ee_mass"] = ak.to_numpy(ak.flatten(wz_data_df["ee_mass"]))
diboson_data_df["ee_mass"] = ak.to_numpy(ak.flatten(diboson_data_df["ee_mass"]))
vjets_data_df["ee_mass"] = ak.to_numpy(ak.flatten(vjets_data_df["ee_mass"]))
other_data_df["ee_mass"] = ak.to_numpy(ak.flatten(other_data_df["ee_mass"]))



plot_data(
    np.hstack(main_analysis_df["ee_mass"]),  # Flatten Z_mass column
    75, 105, 1.5, r"$M_{ee}$ [GeV]", 'events / bin',
    np.hstack(wz_data_df["ee_mass"]), np.hstack(wz_data_df['Weight']),
    np.hstack(diboson_data_df["ee_mass"]), np.hstack(diboson_data_df['Weight']),
    np.hstack(vjets_data_df["ee_mass"]), np.hstack(vjets_data_df['Weight']),
    np.hstack(other_data_df["ee_mass"]), np.hstack(other_data_df['Weight'])
)